In [1]:
import pandas as pd
import numpy as np
import datetime
import json
import glob
from fuzzywuzzy import fuzz
from haversine import haversine
# "S:\Smoothie King\Research\Store List\Updated RAF List - Number of Opened Stores-2019-09-13.xlsx"

date_dig_today=datetime.datetime.now().date() #dig short for interger digit
date_dig_today=date_dig_today-datetime.timedelta(days=date_dig_today.weekday()) # Monday
date_dig_today=str(date_dig_today)
date_dig_today=date_dig_today.split("-")[0]+date_dig_today.split("-")[1]+date_dig_today.split("-")[2]
date_dig_today

/usr/local/lib/python3.6/site-packages/fuzzywuzzy/fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


'20191202'

In [2]:
new_store_list=pd.read_csv("/home/jian/Projects/Smoothie_King/weekly_TA_updates/output_"+date_dig_today+"/new_store_appared.csv",dtype=str)
new_store_list_store_num_df=new_store_list[['storenumber']]

df_recent_store_list=pd.read_excel("/home/jian/Projects/Smoothie_King/store_list_weekly/StoreList_20191202.xlsx",dtype=str)

new_store_list_add_available=pd.merge(new_store_list_store_num_df,df_recent_store_list,on="storenumber",how="left")
new_store_list_add_available=new_store_list_add_available[~pd.isnull(new_store_list_add_available['zip'])]
new_store_list_add_not_available=new_store_list[~new_store_list['storenumber'].isin(new_store_list_add_available['storenumber'])]

new_store_list=new_store_list_add_available.append(new_store_list_add_not_available)
new_store_list_detailed=new_store_list.copy()

In [3]:
new_store_list['address_str']=new_store_list['Address']+", "+new_store_list['city']+", "+new_store_list['state']+", "+new_store_list['zip']
'''
new_store_list_SOTF_forced=new_store_list[pd.isnull(new_store_list['address_str'])]
new_store_list_SOTF_forced['address_str']=new_store_list_SOTF_forced['Address']+", "+new_store_list_SOTF_forced['city']+", "+new_store_list_SOTF_forced['state']
new_store_list_SOTF_forced=new_store_list_SOTF_forced[['storenumber','address_str']].drop_duplicates()
new_store_list_SOTF_forced
'''

new_store_list=new_store_list[['storenumber','address_str']].drop_duplicates()
new_store_list=new_store_list[~pd.isnull(new_store_list['address_str'])]
new_store_list

,storenumber,address_str
0,1539,"1501-A Cornerside Boulevard, Vienna, VA, 22182"
1,1747,"1010-H BEARDS HILL RD., Aberdeen, MD, 21001"
2,1671,"8361 W. Sunrise Blvd, Sunrise, FL, 33322"
3,1707,"105 Kennings Road,Suite 105-2A, Crosby, TX, 77532"
4,1802,"18602 Kuykendahl Rd., Suite #400, Spring, TX, ..."
5,1789,"1309 W FairMonth Pkwy,Suite O, La Porte, TX, 7..."
6,1797,"1659 STATE HIGHWAY 46 W, SUITE 175, New Braunf..."
7,1768,"1100 W Main St, Lewisville, TX, 75067"


In [4]:
latest_SG_brand_info=pd.read_csv("/home/jian/Docs/SafeGraph/places_20191202/brand_info/brand_info.csv",
                                dtype=str,sep=",")
[x for x in latest_SG_brand_info['brand_name'].unique().tolist() if "moothie" in x]

['Emerald City Smoothie',
 'Mr. Smoothie',
 'Planet Smoothie',
 'Pulp Juice and Smoothie Bar',
 'Smoothie Factory',
 'Smoothie King',
 'Tropical Smoothie Café']

In [5]:
POI_data=glob.glob("/home/jian/Docs/SafeGraph/places_20191202/poi/*csv.gz")

In [6]:
all_SG_SK_POI=pd.DataFrame()

for file in POI_data:
    df=pd.read_csv(file,dtype=str,compression="gzip")
    df=df[df['brands']=="Smoothie King"]
    all_SG_SK_POI=all_SG_SK_POI.append(df)
    print(datetime.datetime.now(),file)

2019-12-03 10:26:51.815171 /home/jian/Docs/SafeGraph/places_20191202/poi/core_poi-geometry-part9.csv.gz
2019-12-03 10:27:23.085373 /home/jian/Docs/SafeGraph/places_20191202/poi/core_poi-geometry-part10.csv.gz
2019-12-03 10:27:52.393955 /home/jian/Docs/SafeGraph/places_20191202/poi/core_poi-geometry-part7.csv.gz
2019-12-03 10:28:03.826604 /home/jian/Docs/SafeGraph/places_20191202/poi/core_poi-geometry-part5.csv.gz
2019-12-03 10:28:17.291472 /home/jian/Docs/SafeGraph/places_20191202/poi/core_poi-geometry-part2.csv.gz
2019-12-03 10:28:25.520064 /home/jian/Docs/SafeGraph/places_20191202/poi/core_poi-geometry-part8.csv.gz
2019-12-03 10:28:35.960645 /home/jian/Docs/SafeGraph/places_20191202/poi/core_poi-geometry-part1.csv.gz
2019-12-03 10:28:54.246104 /home/jian/Docs/SafeGraph/places_20191202/poi/core_poi-geometry-part3.csv.gz
2019-12-03 10:29:41.587833 /home/jian/Docs/SafeGraph/places_20191202/poi/core_poi-geometry-part6.csv.gz
2019-12-03 10:30:19.466033 /home/jian/Docs/SafeGraph/places_201

In [9]:
pd.options.display.max_columns=99
all_SG_SK_POI=all_SG_SK_POI.reset_index()
del all_SG_SK_POI['index']
print(all_SG_SK_POI.shape)
all_SG_SK_POI.head(2)

(997, 21)


,safegraph_place_id,parent_safegraph_place_id,safegraph_brand_ids,location_name,brands,top_category,sub_category,naics_code,latitude,longitude,street_address,city,region,postal_code,open_hours,polygon_wkt,polygon_class,phone_number,is_synthetic,includes_parking_lot,iso_country_code
0,sg:025679727d7a415081ce9689cf79d830,NaN,SG_BRAND_19800a6b4d6fb62e6752413f3ac722ef,Smoothie King,Smoothie King,Restaurants and Other Eating Places,Snack and Nonalcoholic Beverage Bars,722515,39.802209,-94.815677,3508 N Belt Hwy,Saint Joseph,MO,64506,"{ ""Mon"": [[""7:00"", ""21:00""]], ""Tue"": [[""7:00"",...",POLYGON ((-94.81562465429306 39.80225965113186...,SHARED_POLYGON,+18162735511,false,false,US
1,sg:ed478a8a47ad43eba82cf177bf31ef79,NaN,SG_BRAND_19800a6b4d6fb62e6752413f3ac722ef,Smoothie King,Smoothie King,Restaurants and Other Eating Places,Snack and Nonalcoholic Beverage Bars,722515,39.432399,-76.623031,1830 York Rd Ste C,Timonium,MD,21093,"{ ""Mon"": [[""7:00"", ""21:00""]], ""Tue"": [[""7:00"",...",POLYGON ((-76.62328489708739 39.43232522635230...,OWNED_POLYGON,+14103084930,false,false,US


In [10]:
all_SG_SK_POI['SG_address_str']=all_SG_SK_POI['street_address']+", "+all_SG_SK_POI['city']+", "+\
                                all_SG_SK_POI['region']+", "+all_SG_SK_POI['postal_code']

In [11]:
def match_best_str(str_value,list_2):
    str_value=str_value.lower()
    ratio_max=0
    best_match=""
    for SG_add in list_2:
        ratio=fuzz.partial_ratio(str_value,SG_add)
        if ratio>ratio_max:
            ratio_max=ratio
            best_match=SG_add
    return [best_match,ratio_max]

In [12]:
all_SG_SK_POI['matched_new_stores']=np.nan
all_SG_SK_POI['highest_score']=0
for i in range(len(all_SG_SK_POI)):
    max_ratio=0
    best_match=np.nan
    store_SG=all_SG_SK_POI['SG_address_str'][i]
    for new_store_add in new_store_list['address_str'].tolist():
        new_ratio=fuzz.partial_ratio(store_SG,new_store_add.lower())
        if new_ratio>max_ratio:
            max_ratio=new_ratio
            best_match=new_store_add
            
    all_SG_SK_POI['matched_new_stores'][i]=best_match
    all_SG_SK_POI['highest_score'][i]=max_ratio

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/home/jian/.local/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [13]:
match_65=all_SG_SK_POI[all_SG_SK_POI['highest_score']>=65][['SG_address_str','matched_new_stores','highest_score','latitude','longitude']]

for ind,row in match_65.iterrows():
    print(ind,row['SG_address_str']," | ",row['matched_new_stores'])

31 1309 W Fairmont Pkwy Ste D, La Porte, TX, 77571  |  1309 W FairMonth Pkwy,Suite O, La Porte, TX, 77571
181 8361 W Sunrise Blvd, Plantation, FL, 33322  |  8361 W. Sunrise Blvd, Sunrise, FL, 33322
346 1659 W State Highway 46 Ste 175, New Braunfels, TX, 78132  |  1659 STATE HIGHWAY 46 W, SUITE 175, New Braunfels, TX, 78132
558 1010 Beards Hill Rd, Aberdeen, MD, 21001  |  1010-H BEARDS HILL RD., Aberdeen, MD, 21001
876 105 Kennings Rd # 105-2A, Crosby, TX, 77532  |  105 Kennings Road,Suite 105-2A, Crosby, TX, 77532
922 1501a Cornerside Boulevard, Vienna, VA, 22182  |  1501-A Cornerside Boulevard, Vienna, VA, 22182


In [14]:
# All good
match_65=match_65.loc[[31,181,346,558,876,922],:]
match_65

,SG_address_str,matched_new_stores,highest_score,latitude,longitude
31,"1309 W Fairmont Pkwy Ste D, La Porte, TX, 77571","1309 W FairMonth Pkwy,Suite O, La Porte, TX, 7...",72,29.651361,-95.031523
181,"8361 W Sunrise Blvd, Plantation, FL, 33322","8361 W. Sunrise Blvd, Sunrise, FL, 33322",68,26.148596,-80.262615
346,"1659 W State Highway 46 Ste 175, New Braunfels...","1659 STATE HIGHWAY 46 W, SUITE 175, New Braunf...",79,29.713163,-98.160644
558,"1010 Beards Hill Rd, Aberdeen, MD, 21001","1010-H BEARDS HILL RD., Aberdeen, MD, 21001",78,39.52124,-76.181654
876,"105 Kennings Rd # 105-2A, Crosby, TX, 77532","105 Kennings Road,Suite 105-2A, Crosby, TX, 77532",70,29.889938,-95.062566
922,"1501a Cornerside Boulevard, Vienna, VA, 22182","1501-A Cornerside Boulevard, Vienna, VA, 22182",87,38.929557,-77.244659


In [15]:
match_65=match_65.loc[[31,181,346,558,876,922],:]
new_store_list_SG=pd.merge(new_store_list,match_65,left_on=['address_str'],right_on=['matched_new_stores'],how="left")
# new_store_list_SG['highest_score']=np.nan # no good and remove
new_store_list_new=new_store_list_SG[pd.isnull(new_store_list_SG['highest_score'])]
new_store_list_SG=new_store_list_SG[pd.notnull(new_store_list_SG['highest_score'])]
new_store_list_SG

,storenumber,address_str,SG_address_str,matched_new_stores,highest_score,latitude,longitude
0,1539,"1501-A Cornerside Boulevard, Vienna, VA, 22182","1501a Cornerside Boulevard, Vienna, VA, 22182","1501-A Cornerside Boulevard, Vienna, VA, 22182",87.0,38.929557,-77.244659
1,1747,"1010-H BEARDS HILL RD., Aberdeen, MD, 21001","1010 Beards Hill Rd, Aberdeen, MD, 21001","1010-H BEARDS HILL RD., Aberdeen, MD, 21001",78.0,39.52124,-76.181654
2,1671,"8361 W. Sunrise Blvd, Sunrise, FL, 33322","8361 W Sunrise Blvd, Plantation, FL, 33322","8361 W. Sunrise Blvd, Sunrise, FL, 33322",68.0,26.148596,-80.262615
3,1707,"105 Kennings Road,Suite 105-2A, Crosby, TX, 77532","105 Kennings Rd # 105-2A, Crosby, TX, 77532","105 Kennings Road,Suite 105-2A, Crosby, TX, 77532",70.0,29.889938,-95.062566
5,1789,"1309 W FairMonth Pkwy,Suite O, La Porte, TX, 7...","1309 W Fairmont Pkwy Ste D, La Porte, TX, 77571","1309 W FairMonth Pkwy,Suite O, La Porte, TX, 7...",72.0,29.651361,-95.031523
6,1797,"1659 STATE HIGHWAY 46 W, SUITE 175, New Braunf...","1659 W State Highway 46 Ste 175, New Braunfels...","1659 STATE HIGHWAY 46 W, SUITE 175, New Braunf...",79.0,29.713163,-98.160644


In [16]:
print(new_store_list_SG.shape)
new_store_list_SG=new_store_list_SG[['storenumber','latitude','longitude']].rename(columns={"latitude":"assigned_lat","longitude":"assigned_long"})
new_store_list_SG=pd.merge(new_store_list_detailed,new_store_list_SG,on="storenumber",how="left")
new_store_list_SG=new_store_list_SG[pd.notnull(new_store_list_SG['assigned_lat'])]
new_store_list_SG=new_store_list_SG.reset_index()
del new_store_list_SG['index']
new_store_list_SG['dist']=np.nan
new_store_list_SG['QC_level']=np.nan

for ind,row in new_store_list_SG.iterrows():
    client_cent=(float(row['Latitude']),float(row['Longitude']))
    SG_cent=(float(row['assigned_lat']),float(row['assigned_long']))
    new_store_list_SG.loc[ind,'dist']=haversine(client_cent,SG_cent)*1000
new_store_list_SG['QC_level']=np.where(new_store_list_SG['dist']<=30,"POI_address_matched_in_30_meters","POI_address_matched_above_30_meters")
new_store_list_SG['assigned_source']=np.where(new_store_list_SG['dist']<=30,"Client_list","POI_data")
new_store_list_SG['assigned_lat']=np.where(new_store_list_SG['dist']<=30,new_store_list_SG['Latitude'],new_store_list_SG['assigned_lat'])
new_store_list_SG['assigned_long']=np.where(new_store_list_SG['dist']<=30,new_store_list_SG['Longitude'],new_store_list_SG['assigned_long'])
new_store_list_SG['updated_date']=str(datetime.datetime.now().date())

(6, 7)


In [18]:
new_store_list_new

,storenumber,address_str,SG_address_str,matched_new_stores,highest_score,latitude,longitude
4,1802,"18602 Kuykendahl Rd., Suite #400, Spring, TX, ...",NaN,NaN,NaN,NaN,NaN
7,1768,"1100 W Main St, Lewisville, TX, 75067",NaN,NaN,NaN,NaN,NaN


In [20]:
new_store_list_new.set_index("storenumber").to_dict()['address_str']

{'1768': '1100 W Main St, Lewisville, TX, 75067',
 '1802': '18602 Kuykendahl Rd., Suite #400, Spring, TX, 77379'}

In [21]:
google_dict={}

# 1768	1100 W Main St, Lewisville, TX, 75067
google_dict.update({"1768":{"Google_Add":"1100 W Main St, Lewisville, TX 75067","Google_Coor":[33.0417436,-97.0227896]}})

# 1802	18602 Kuykendahl Rd., Suite #400, Spring, TX, 77379
google_dict.update({"1802":{"Google_Add":"18602 Kuykendahl Rd, Spring, TX 77379","Google_Coor":[30.0408382,-95.4885916]}})

# "QC_level","assigned_lat","assigned_long","assigned_source","updated_date","SOTF_Forced"

# new_store_list_detailed

In [22]:
# new_store_list=pd.read_csv("/home/jian/Projects/Smoothie_King/weekly_TA_updates/output_"+date_dig_today+"/new_store_appared.csv",dtype=str)
new_store_list=new_store_list_detailed.copy()
new_store_list=new_store_list[new_store_list['storenumber'].isin(list(google_dict.keys()))]
new_store_list['Latitude']=new_store_list['Latitude'].astype(float)
new_store_list['Longitude']=new_store_list['Longitude'].astype(float)
new_store_list['dist']=np.nan
soft_list=[]
new_store_list=new_store_list.reset_index()
del new_store_list['index']
for i in range(len(new_store_list)):
    
    client_coor=[new_store_list['Latitude'][i],new_store_list['Longitude'][i]]
    if pd.notnull(new_store_list['Latitude'][i]):
        Google_coor=google_dict[new_store_list['storenumber'][i]]['Google_Coor']
        dist=haversine(client_coor,Google_coor)*1000
        new_store_list['dist'][i]=dist
    
    else:
        soft_list=soft_list+[new_store_list['storenumber'][i]]
        Google_coor=google_dict[new_store_list['storenumber'][i]]['Google_Coor']
        dist=np.nan
        new_store_list['dist'][i]=dist
        
new_store_list

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


,storenumber,Address,city,state,zip,StoreOpenDate,Latitude,Longitude,Ownership,Franchisee_Name,IPAddress,DriveThru,status,StoreCloseDate,dist
0,1802,"18602 Kuykendahl Rd., Suite #400",Spring,TX,77379,2019-10-26 00:00:00,30.04093,-95.48840,Franchised,Golda Fondal,NaN,Y,New,NaN,21.079451
1,1768,1100 W Main St,Lewisville,TX,75067,2019-11-07 00:00:00,33.04075,-97.02266,Franchised,Jackie Ben-Meir,NaN,Y,New,NaN,111.141905


In [23]:
new_store_list['QC_level']=np.where(new_store_list['dist']<=30,"Google_in_30_meters",np.where(new_store_list['dist']<=100,"Google_30_to_100_meters","Google_above_100_meters"))

new_store_list_1=new_store_list[new_store_list['dist']<=30]
new_store_list_2=new_store_list[(new_store_list['dist']>30) | pd.isnull(new_store_list['dist'])]

new_store_list_1['assigned_lat']=new_store_list_1['Latitude']
new_store_list_1['assigned_long']=new_store_list_1['Longitude']
new_store_list_1['assigned_source']="Client_list"



new_store_list_2['assigned_lat']=new_store_list_2['storenumber'].apply(lambda x: google_dict[x]['Google_Coor'][0])
new_store_list_2['assigned_long']=new_store_list_2['storenumber'].apply(lambda x: google_dict[x]['Google_Coor'][1])
new_store_list_2['assigned_source']="Google_Map"

new_store_list=new_store_list_1.append(new_store_list_2)

new_store_list['updated_date']=str(datetime.datetime.now().date())

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas

In [24]:
new_store_list=new_store_list.append(new_store_list_SG)

/home/jian/.local/lib/python3.6/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [25]:
import os

os.getcwd()

'/home/jian/Projects/Smoothie_King/weekly_TA_updates'

In [26]:
col_order=pd.read_excel("/home/jian/Projects/Smoothie_King/weekly_TA_updates/output_20190429/new_store_updates_2019-04-29.xlsx",nrows=10)
col_order=col_order.columns.tolist()
new_store_list=new_store_list[col_order]

In [27]:
writer=pd.ExcelWriter("/home/jian/Projects/Smoothie_King/weekly_TA_updates/output_"+date_dig_today+"/new_store_updates_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")

new_store_list.to_excel(writer,"New_Stores",index=False)
writer.save()

In [28]:
new_store_list

,storenumber,Address,city,state,zip,StoreOpenDate,Latitude,Longitude,Ownership,Franchisee_Name,IPAddress,DriveThru,status,StoreCloseDate,QC_level,assigned_lat,assigned_long,assigned_source,updated_date,dist
0,1802,"18602 Kuykendahl Rd., Suite #400",Spring,TX,77379,2019-10-26 00:00:00,30.0409,-95.4884,Franchised,Golda Fondal,NaN,Y,New,NaN,Google_in_30_meters,30.0409,-95.4884,Client_list,2019-12-03,21.079451
1,1768,1100 W Main St,Lewisville,TX,75067,2019-11-07 00:00:00,33.0408,-97.0227,Franchised,Jackie Ben-Meir,NaN,Y,New,NaN,Google_above_100_meters,33.0417,-97.0228,Google_Map,2019-12-03,111.141905
0,1539,1501-A Cornerside Boulevard,Vienna,VA,22182,2019-10-11 00:00:00,38.929800,-77.244600,Company,NaN,NaN,N,New,NaN,POI_address_matched_in_30_meters,38.929800,-77.244600,Client_list,2019-12-03,27.498152
1,1747,1010-H BEARDS HILL RD.,Aberdeen,MD,21001,2019-10-28 00:00:00,39.521990,-76.184380,Franchised,John Avalos,NaN,N,New,NaN,POI_address_matched_above_30_meters,39.52124,-76.181654,POI_data,2019-12-03,248.247692
2,1671,8361 W. Sunrise Blvd,Sunrise,FL,33322,2019-11-14 00:00:00,26.148320,-80.262340,Franchised,Millie Medrano,NaN,N,New,NaN,POI_address_matched_above_30_meters,26.148596,-80.262615,POI_data,2019-12-03,41.174245
3,1707,"105 Kennings Road,Suite 105-2A",Crosby,TX,77532,2019-10-22 00:00:00,29.888720,-95.061640,Franchised,Candra Calvin,NaN,N,New,NaN,POI_address_matched_above_30_meters,29.889938,-95.062566,POI_data,2019-12-03,162.210100
4,1789,"1309 W FairMonth Pkwy,Suite O",La Porte,TX,77571,2019-10-30 00:00:00,29.651590,-95.031960,Franchised,Charles (Drew) Coxie,NaN,N,New,NaN,POI_address_matched_above_30_meters,29.651361,-95.031523,POI_data,2019-12-03,49.312255
5,1797,"1659 STATE HIGHWAY 46 W, SUITE 175",New Braunfels,TX,78132,2019-11-04 00:00:00,29.713790,-98.161030,Franchised,Paul Flores,NaN,N,New,NaN,POI_address_matched_above_30_meters,29.713163,-98.160644,POI_data,2019-12-03,79.059577
